In [47]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from skimage.feature import local_binary_pattern

In [48]:
RADIUS = 1
N_POINTS = 8 * RADIUS
METHOD = "uniform"

In [49]:
def extract_lbp_features(image):
    lbp = local_binary_pattern(image, N_POINTS, RADIUS, METHOD)
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, N_POINTS + 3), range=(0, N_POINTS + 2))
    hist = hist.astype("float")
    hist /= hist.sum()  # Normalisasi
    return hist


In [50]:
def detect_faces(image, face_cascade):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return faces, gray

In [51]:
def load_dataset(dataset_path, face_cascade):
    data = []
    labels = []

    for person in os.listdir(dataset_path):
        person_path = os.path.join(dataset_path, person)
        if os.path.isdir(person_path):
            for img_file in os.listdir(person_path):
                img_path = os.path.join(person_path, img_file)
                image = cv2.imread(img_path)
                if image is None:
                    print(f"Failed to read image: {img_path}")
                    continue

                faces, gray = detect_faces(image, face_cascade)
                for (x, y, w, h) in faces:
                    face = gray[y:y+h, x:x+w]
                    face = cv2.resize(face, (128, 128))  # Resize untuk konsistensi
                    features = extract_lbp_features(face)
                    data.append(features)
                    labels.append(person)
    return data, labels

In [52]:
def train_model(data, labels):
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
    model = SVC(kernel="linear", probability=True)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")
    return model

In [53]:
# Tampilkan gambar dengan ukuran yang disesuaikan
def show_resized_image(window_name, image, width=200):
    """Tampilkan gambar dengan lebar yang disesuaikan."""
    height, original_width = image.shape[:2]
    scaling_factor = width / original_width
    new_height = int(height * scaling_factor)
    resized_image = cv2.resize(image, (width, new_height))
    cv2.imshow(window_name, resized_image)

In [54]:
# Program utama
if __name__ == "__main__":
    # Path ke folder dataset
    dataset_path = "dataset/"

    # Load cascade untuk deteksi wajah
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

    # Load dataset
    print("Loading dataset...")
    data, labels = load_dataset(dataset_path, face_cascade)

    if not data or not labels:
        print("Dataset is empty or not found. Please check your dataset path and structure.")
        exit()

    # Train model
    print("Training model...")
    model = train_model(data, labels)

    # Test dengan gambar baru
    test_image_path = "testimage2.jpeg"  # Ganti dengan gambar uji
    test_image = cv2.imread(test_image_path)
    if test_image is None:
        print(f"Failed to read test image: {test_image_path}")
        exit()

    print("Testing with new image...")
    faces, gray = detect_faces(test_image, face_cascade)
    for (x, y, w, h) in faces:
        face = gray[y:y+h, x:x+w]
        face = cv2.resize(face, (128, 128))
        features = extract_lbp_features(face)
        prediction = model.predict([features])
        print(f"Predicted: {prediction[0]}")
        cv2.rectangle(test_image, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(test_image, prediction[0], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Tampilkan hasil
    show_resized_image("Result", test_image, width=500)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Loading dataset...
Training model...
Accuracy: 100.00%
Testing with new image...
Predicted: Pak Jokowi
